In [3]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np

In [ ]:
learning_rate = 1e-4
keep_prob_rate = 0.7 #
max_epoch = 3
BATCH_SIZE = 50
DOWNLOAD_MNIST = True

In [4]:
train_data = torchvision.datasets.MNIST(root='./data',train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset = train_data ,batch_size= BATCH_SIZE ,shuffle= True)

test_data = torchvision.datasets.MNIST(root = './data',train = False)
test_x = torch.unsqueeze(test_data.data, dim=1)[:500] / 255.
test_y = test_data.targets[:500].numpy()

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(  # ???
                # patch 7 * 7 ; 1  in channels ; 32 out channels ; ; stride is 1
                # padding style is same(that means the convolution opration's input and output have the same size)
                in_channels=1,
                out_channels=32,
                kernel_size=7,
                stride=1,
                padding=3,
            ),
            nn.ReLU(),  # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.conv2 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.out1 = nn.Linear(7 * 7 * 64, 1024, bias=True)  # full connection layer one

        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024, 10, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(
            x.size(0), -1
        )  # flatten the output of coonv2 to (batch_size ,32 * 7 * 7)    # ???
        out1 = self.out1(x)
        out1 = F.relu(out1)
        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2, dim=-1)
        return output

In [8]:
def test(cnn):
    y_pre = cnn(test_x)
    _, pre_index = torch.max(y_pre, 1)
    pre_index = pre_index.view(-1)
    prediction = pre_index.data.numpy()
    correct = np.sum(prediction == test_y)
    return correct / y_pre.size(0)


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss()
    for _ in range(max_epoch):
        for step, (x, y) in enumerate(train_loader):
            output = cnn(x)
            loss = loss_func(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if step != 0 and step % 50 == 0:
                print(
                    "=" * 10,
                    step,
                    "=" * 5,
                    "=" * 5,
                    "test accuracy is ",
                    test(cnn),
                    "=" * 10,
                )

In [9]:
if __name__ == '__main__':
    cnn = CNN()
    train(cnn)

========== 50 ===== ===== test accuracy is  0.426 ==========
========== 100 ===== ===== test accuracy is  0.642 ==========
========== 150 ===== ===== test accuracy is  0.738 ==========
========== 200 ===== ===== test accuracy is  0.814 ==========
========== 250 ===== ===== test accuracy is  0.856 ==========
========== 300 ===== ===== test accuracy is  0.87 ==========
========== 350 ===== ===== test accuracy is  0.84 ==========
========== 400 ===== ===== test accuracy is  0.904 ==========
========== 450 ===== ===== test accuracy is  0.894 ==========
========== 500 ===== ===== test accuracy is  0.896 ==========
========== 550 ===== ===== test accuracy is  0.918 ==========
========== 600 ===== ===== test accuracy is  0.906 ==========
========== 650 ===== ===== test accuracy is  0.918 ==========
========== 700 ===== ===== test accuracy is  0.918 ==========
========== 750 ===== ===== test accuracy is  0.938 ==========
========== 800 ===== ===== test accuracy is  0.918 ==========
========== 